In [1]:
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
def build_model(hp):
    units = hp.Int(name='units', min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation='relu'),
        layers.Dense(10, activation='softmax'),
    ])
    optimizer = hp.Choice(name='optimizer', values=['rmsprop', 'adam'])
    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [5]:
!pip install keras-tuner -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.0/98.0 KB 837.2 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 MB 3.9 MB/s eta 0:00:0000:0100:01


In [6]:
import kerastuner as kt

/var/folders/_f/s_khv65n16s45b6z9py_kgw00000gn/T/ipykernel_32854/1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [7]:
class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes
    
    def build(self, hp):
        units = hp.Int(name='units', min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation='relu'),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        optimizer = hp.Choice(name='optimizer', values=['rmsprop', 'adam'])
        model.compile(optimizer=optimizer,
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model
    
hypermodel = SimpleMLP(num_classes=10)

In [14]:
tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',
    max_trials=100,
    executions_per_trial = 2,
    directory='mnist_kt_test',
    overwrite=True)

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [10]:
(x_train, y_train),(x_test, y_test) = keras.datasets.mnist.load_data()

In [ ]:
x_train = x_train.reshape((-1,28*28)).astype('float32')/255
x_test = x_test.reshape((-1,28*28)).astype('float32')/255
x_train_full = x_train[:]
Y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
]
tuner.search(
    x_train, y_train,
    batch_size = 128,
    epochs = 100,
    validation_data = (x_val, y_val),
    callbacks = callbacks,
    verbose=2)

In [15]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [ ]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss', mode='min', patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data = (x_val, y_val),
        epochs = 100,
        batch_size = 128,
        callbacks = callbacks)
    val_loss_per_epoch = history.history['val_loss']
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch))+1
    print(f'Best epoch: {best_epoch}')
    return best_epoch

In [ ]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(x_train_full, y_train_fll, batch_size = 128, epochs = int(best_epoch * 1.2))
    return model

best_model = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_model.append(model)